In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import sys; sys.path.append('../..') ; sys.path.append('..') ;
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
# dummy trainloader
trainloader = data.DataLoader(data.TensorDataset(torch.Tensor(1), torch.Tensor(1)), batch_size=1, shuffle=True)
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import random
seed = 0
np.random.seed(seed)
torch.random.manual_seed(seed)
random.seed(seed)

In [ ]:
reviews_df = pd.read_csv('/content/drive/MyDrive/Python/sandhiya/CSE573-Fake-Review-Detection-Group-6-main/CSE573-Fake-Review-Detection-Group-6-main/CODE/final_data.csv')
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(reviews_df.shape[0]))

# Display 10 random rows from the data.
reviews_df.sample(10)

Number of training sentences: 40,432



,Unnamed: 0,category,rating,label,text_,lemmatized_text,stemmed_text,cleaned_text,CleanedText,sentiment,...,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40
7690,7690,Sports_and_Outdoors_5,3.0,OR,"i initially mounted this scope on my ar, and h...","i initially mount this scope on my ar, and hav...","i initi mount thi scope on my ar, and have use...",i initi mount thi scope on my ar and have use ...,initi mount thi scope ar use half dozen outing...,Positive,...,-0.172851,-1.490920,-0.402284,0.887081,-2.557670,-1.641267,1.772270,1.093240,0.130956,-0.705495
23859,23859,Kindle_Store_5,5.0,OR,aiden james the judas chronicles are always ...,aiden jam the juda chronicle be always a good ...,aiden jam the juda chronicl be alway a good re...,aiden jam the juda chronicl be alway a good re...,aiden jam juda chronicl alway good read well w...,Positive,...,-0.010784,-0.002897,-0.444728,0.057847,0.159042,-0.077125,0.125905,-0.014327,-0.212609,0.027911
19138,19138,Tools_and_Home_Improvement_5,4.0,CG,these locks are very rugged and have the quali...,these lock be very rugged and have the quality...,these lock be veri rug and have the qualiti an...,these lock be veri rug and have the qualiti an...,lock veri rug qualiti durabl steel lock one ba...,Positive,...,1.890627,0.468183,-1.378723,0.012022,-1.965816,3.190603,0.304400,-0.010597,-1.362760,0.215563
11714,11714,Electronics_5,1.0,OR,2013-08-04 update - they worked great for abou...,2013-08-04 update - they work great for about ...,2013-08-04 updat - they work great for about a...,2013-08-04 updat - they work great for about a...,2013-08-04 updat - work great month two week a...,Positive,...,0.839335,0.640123,0.139822,1.405355,-0.186191,-0.288932,0.507382,-1.068334,-0.670127,-1.286942
36539,36539,Toys_and_Games_5,5.0,OR,i have spent some amount of time playing with ...,i have spend some amount of time play with thi...,i have spend some amount of time play with thi...,i have spend some amount of time play with thi...,spend amount time play thi walnut size wonder ...,Positive,...,0.442747,-0.232406,-2.340552,-1.198724,2.371439,-0.415509,-2.334078,-0.107667,-0.306078,0.294982
11964,11964,Electronics_5,5.0,CG,seagate has long been a reliable source of rel...,seagate have long be a reliable source of reli...,seagat have long be a reliabl sourc of reliabl...,seagat have long be a reliabl sourc of reliabl...,seagat long reliabl sourc reliabl data use thi...,Positive,...,5.483281,2.734244,-2.353126,-1.520309,-1.835619,4.092201,-0.014219,-2.034539,2.864111,-2.773597
8566,8566,Electronics_5,5.0,OR,it was exactly what i needed to connect the se...,it be exactly what i need to connect the secon...,it be exactli what i need to connect the secon...,it be exactli what i need to connect the secon...,exactli need connect second optic drive instal...,Positive,...,-0.286526,0.557833,-0.102856,0.147671,0.177544,0.155852,-0.024068,0.177522,-0.458071,0.385681
38651,38651,Clothing_Shoes_and_Jewelry_5,5.0,CG,it's always hard finding a pair of shoes with ...,it's always hard find a pair of shoe with the ...,it' alway hard find a pair of shoe with the ri...,it alway hard find a pair of shoe with the rig...,alway hard find pair shoe right fit onli one w...,Positive,...,0.335455,-0.041342,-0.255823,0.030219,0.029752,-0.369946,-0.305590,-0.147820,0.270003,-0.273654
16444,16444,Tools_and_Home_Improvement_5,5.0,CG,i got these for my son to wear as a night-visi...,i get these for my son to wear a a night-visio...,i get these for my son to wear a a night-vis g...,i get these for my son to wear a a night-vis g...,get son wear night-vis goggl coupl,Neutral,...,-0.282724,-0.000430,-0.377462,0.483637,-0.034491,0.039426,-0.218876,1.417872,0.924180,0.235724
154,154,Home_and_Kitchen_5,5.0,CG,perfect! works in your oven and has the right ...,perfect! work in your oven and have the right ...,perfect! work in your oven and have the right ...,perfect work in your oven and have the right h...,perfect work oven right height also love rem

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Label encoding for labels
encoder = LabelEncoder()
encoder.fit(reviews_df['label'])
reviews_df['label'] = encoder.transform(reviews_df['label'])

# Select only categorical features
categorical_features = reviews_df[['category', 'sentiment', 'semantic_relevance']]


In [ ]:
#one hot encoding
one_hot_encoded = pd.get_dummies(categorical_features, prefix=['category', 'sentiment', 'semantic_relevance'])
pca_columns = reviews_df.filter(regex='^PC\d+$')
pc_columns = [f'PC{i}' for i in range(1, 41)]
pca_columns[pc_columns] = pca_columns[pc_columns].apply(pd.to_numeric, errors='coerce')
pca_columns[pc_columns] = pca_columns[pc_columns].fillna(pca_columns[pc_columns].mean())

<ipython-input-7-04e5007b34d7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pca_columns[pc_columns] = pca_columns[pc_columns].apply(pd.to_numeric, errors='coerce')
<ipython-input-7-04e5007b34d7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pca_columns[pc_columns] = pca_columns[pc_columns].fillna(pca_columns[pc_columns].mean())


In [ ]:
df_with_PCA = pd.concat([one_hot_encoded, pca_columns, reviews_df['label']], axis=1)
df_with_PCA.head()

,category_Books_5,category_Clothing_Shoes_and_Jewelry_5,category_Electronics_5,category_Home_and_Kitchen_5,category_Kindle_Store_5,category_Movies_and_TV_5,category_Pet_Supplies_5,category_Sports_and_Outdoors_5,category_Tools_and_Home_Improvement_5,category_Toys_and_Games_5,...,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,label
0,False,False,False,True,False,False,False,False,False,False,...,-0.302449,-0.723997,-0.285891,0.147164,-0.073054,0.450939,-0.130718,0.071797,-0.111493,0
1,False,False,False,True,False,False,False,False,False,False,...,0.091364,-0.448103,-0.067810,0.152155,0.100008,0.200941,-0.138285,0.223359,0.312850,0
2,False,False,False,True,False,False,False,False,False,False,...,-0.100469,-0.381030,0.197660,-0.228100,0.042760,0.085769,0.013475,-0.113090,-0.188620,0
3,False,False,False,True,False,False,False,False,False,False,...,-0.320948,0.241097,-0.326349,-0.208896,-0.020397,-0.218098,-1.324096,-0.117941,0.312803,0
4,False,False,False,True,False,False,False,False,False,False,...,0.109892,-0.206107,-0.290772,-0.127746,0.631337,0.290962,-0.181045,0.162414,-0.077705,0


In [ ]:
SEED = 0
train = df_with_PCA.sample(frac=0.8, random_state = SEED)
test = df_with_PCA.drop(train.index)

In [ ]:
X_train = train.drop('label', axis=1)  # Features (excluding the 'label' column)
y_train = train['label']
X_test= test.drop('label', axis=1)
y_test= test['label']
X_train.head()

,category_Books_5,category_Clothing_Shoes_and_Jewelry_5,category_Electronics_5,category_Home_and_Kitchen_5,category_Kindle_Store_5,category_Movies_and_TV_5,category_Pet_Supplies_5,category_Sports_and_Outdoors_5,category_Tools_and_Home_Improvement_5,category_Toys_and_Games_5,...,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40
7690,False,False,False,False,False,False,False,True,False,False,...,-0.172851,-1.490920,-0.402284,0.887081,-2.557670,-1.641267,1.772270,1.093240,0.130956,-0.705495
23859,False,False,False,False,True,False,False,False,False,False,...,-0.010784,-0.002897,-0.444728,0.057847,0.159042,-0.077125,0.125905,-0.014327,-0.212609,0.027911
19138,False,False,False,False,False,False,False,False,True,False,...,1.890627,0.468183,-1.378723,0.012022,-1.965816,3.190603,0.304400,-0.010597,-1.362760,0.215563
11714,False,False,True,False,False,False,False,False,False,False,...,0.839335,0.640123,0.139822,1.405355,-0.186191,-0.288932,0.507382,-1.068334,-0.670127,-1.286942
36539,False,False,False,False,False,False,False,False,False,True,...,0.442747,-0.232406,-2.340552,-1.198724,2.371439,-0.415509,-2.334078,-0.107667,-0.306078,0.294982


In [ ]:
y_train.head()

,label
7690,1
23859,1
19138,0
11714,1
36539,1


In [ ]:
class Model(nn.Module):
    def __init__(self,device="cpu"):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(51,48)
        self.linear2 = nn.Linear(48,42)
        self.linear3 = nn.Linear(42,36)
        self.linear4 = nn.Linear(36,28)
        self.linear5 = nn.Linear(28,20)
        self.linear6 = nn.Linear(20,16)
        self.linear7 = nn.Linear(16,12)
        self.linear8 = nn.Linear(12,8)
        self.linear9 = nn.Linear(8,4)
        self.linear10 = nn.Linear(4,1)
    def forward(self, x):
        y = torch.sigmoid(self.linear1(x))
        y = torch.sigmoid(self.linear2(y))
        y = torch.sigmoid(self.linear3(y))
        y = torch.sigmoid(self.linear4(y))
        y = torch.sigmoid(self.linear5(y))
        y = torch.sigmoid(self.linear6(y))
        y = torch.sigmoid(self.linear7(y))
        y = torch.sigmoid(self.linear8(y))
        y = torch.sigmoid(self.linear9(y))
        y = torch.sigmoid(self.linear10(y))
        return y

In [ ]:
model = Model().to(device)
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter value: {param.data}")
    print("=" * 30)

Parameter name: linear1.weight
Parameter value: tensor([[-0.0010,  0.0751, -0.1152,  ..., -0.0621, -0.0051,  0.0896],
        [ 0.1392,  0.0556,  0.0189,  ..., -0.0806,  0.1317,  0.0944],
        [-0.0611, -0.0352, -0.1334,  ..., -0.1310, -0.1182, -0.0284],
        ...,
        [-0.0125,  0.0207,  0.0513,  ...,  0.1357, -0.0040,  0.1223],
        [ 0.0187, -0.0521,  0.1027,  ..., -0.0282, -0.1376,  0.1351],
        [ 0.0681, -0.0838, -0.1022,  ...,  0.1348, -0.0376, -0.0189]],
       device='cuda:0')
Parameter name: linear1.bias
Parameter value: tensor([-0.0858, -0.0694,  0.0308,  0.0590, -0.0554, -0.0515,  0.1109, -0.0721,
        -0.1045, -0.0218,  0.1332,  0.0305,  0.0193, -0.0174,  0.0875, -0.0694,
         0.0949,  0.0331,  0.0156,  0.0193, -0.0864,  0.1040,  0.0735, -0.1095,
        -0.1304, -0.0360,  0.0930, -0.0689,  0.1298, -0.0704, -0.1265,  0.0616,
        -0.0846, -0.0221, -0.0437, -0.0111, -0.1220, -0.1284,  0.0602, -0.0159,
        -0.0845, -0.0036, -0.1278,  0.0372,  0.0

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-6)

In [ ]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
for itr in range(1, 20001):
    optimizer.zero_grad()
    # x0, y = sample_annuli(device=device, n_samples=1024)
    yh = model(X_train_tensor)
    print(yh.shape)
    loss = nn.BCELoss()(yh.squeeze(), y_train_tensor.float())
    print(itr,loss.item())
    #loss_traj[itr-1] = loss.item()
    loss.backward()
    optimizer.step()

In [ ]:

with torch.no_grad():
    device = torch.device("cpu")  # Set the device to CPU
    xs, ys = X_train_tensor.to(device), y_train_tensor.to(device)
    model.to(device)  # Move the model to the same device
    y_pred = model(xs)
    y_pred = torch.where(y_pred > 0.5, torch.tensor(1).to(device), torch.tensor(0).to(device)).flatten()

    acc = (y_pred == ys).float().mean()
    print(acc.item())


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32346x56 and 51x48)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Classification report
print("Classification Report:")
print(classification_report(ys, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      6773
           1       0.96      0.90      0.93      6796

    accuracy                           0.93     13569
   macro avg       0.93      0.93      0.93     13569
weighted avg       0.93      0.93      0.93     13569

